# Step 03: Create Batches

This notebook creates batches and jobs from the loaded configuration.

**Tasks:**
- Verify configuration is loaded and valid
- Identify batch types to create based on configuration data
- Preview databases and job configurations
- Create batches (EDM Creation, etc.)
- Display batch and job summaries

In [ ]:
%load_ext autoreload
%autoreload 2

from helpers.notebook_setup import initialize_notebook_context
from helpers import ux
from helpers.configuration import read_configuration, get_base_portfolios, classify_groupings
from helpers.batch import create_batch, get_batches_for_configuration, delete_batch
from helpers.batch_preview import (
    preview_batch_type, preview_export_to_rdm, display_job_preview,
    trunc, list_preview
)
from helpers.database import execute_query
from helpers.constants import BatchType

## 1) Setup

In [ ]:
# Initialize notebook context and step tracking
context, step = initialize_notebook_context('Step_03_Create_Batches.ipynb')

# Display context
ux.header("Batch Creation")
ux.info(f"Cycle: {context.cycle_name}")
ux.info(f"Stage: {context.stage_name}")
ux.info(f"Step: {context.step_name}")
ux.success(f"✓ Step tracking initialized for '{context.step_name}'")

## 2) Verify Configuration

In [ ]:
# Verify configuration exists and is valid
ux.header("Configuration Verification")

# Get cycle ID
cycle_result = execute_query(
    "SELECT id FROM irp_cycle WHERE cycle_name = %s",
    (context.cycle_name,)
)

if cycle_result.empty:
    raise ValueError(f"Cycle not found: {context.cycle_name}")

cycle_id = int(cycle_result.iloc[0]['id'])  # Convert numpy.int64 to Python int

# Get configuration for this cycle
config_result = execute_query(
    "SELECT id, status, created_ts FROM irp_configuration WHERE cycle_id = %s ORDER BY created_ts DESC LIMIT 1",
    (cycle_id,)
)

if config_result.empty:
    ux.error("✗ No configuration found for this cycle")
    ux.info("Please complete Step 02: Validate Configuration File first")
    raise ValueError("No configuration found for cycle")

config_id = int(config_result.iloc[0]['id'])  # Convert numpy.int64 to Python int
config_status = config_result.iloc[0]['status']
config_created = config_result.iloc[0]['created_ts']

# Verify status is VALID or ACTIVE
if config_status not in ['VALID', 'ACTIVE']:
    ux.error(f"✗ Configuration status is '{config_status}' (expected VALID or ACTIVE)")
    raise ValueError(f"Configuration must be VALID or ACTIVE, found: {config_status}")

# Display configuration summary
config_info = [
    ["Configuration ID", config_id],
    ["Status", config_status],
    ["Created", config_created.strftime('%Y-%m-%d %H:%M:%S')]
]
ux.table(config_info, headers=["Property", "Value"])
ux.success("✓ Configuration verified")

step.log(f"Configuration verified: ID={config_id}, Status={config_status}")

## 3) Identify Batch Types to Create

In [ ]:
# Analyze configuration to determine which batch types are needed
ux.header("Batch Type Identification")

# Read configuration data
config_data = read_configuration(config_id)

# Extract configuration_data JSONB field
configuration_data = config_data.get('configuration_data', {})
metadata = configuration_data.get('Metadata', {})

# Identify batch types based on configuration content
# Order matches step execution: Stage 02 → Stage 03 → Stage 04 → Stage 05 → Stage 06
batch_types_info = []
batch_types_to_create = []

# Get common data needed for multiple batch types
databases = configuration_data.get('Databases', [])
portfolios = configuration_data.get('Portfolios', [])
base_portfolios = get_base_portfolios(portfolios) if portfolios else []
treaties = configuration_data.get('Reinsurance Treaties', [])
analyses = configuration_data.get('Analysis Table', [])
groupings = configuration_data.get('Groupings', [])
edm_version = metadata.get('EDM Data Version')
geocode_version = metadata.get('Geocode Version')
rdm_name = metadata.get('Export RDM Name')

# Initialize for use in preview sections
treaty_edm_combinations = set()
analysis_only_groups = []
rollup_groups = []

# === STAGE 02: Data Extraction ===

# Step 01: Data Extraction (Base Portfolios only - extracts data from SQL Server to CSV)
if base_portfolios:
    batch_types_info.append([BatchType.DATA_EXTRACTION, len(base_portfolios), "One job per base portfolio (SQL extraction to CSV)"])
    batch_types_to_create.append(BatchType.DATA_EXTRACTION)

# === STAGE 03: Data Import ===

# Step 01: EDM Creation (Databases sheet)
if databases:
    batch_types_info.append([BatchType.EDM_CREATION, len(databases), "One job per database"])
    batch_types_to_create.append(BatchType.EDM_CREATION)

# Step 02: Portfolio Creation (Base Portfolios only)
if base_portfolios:
    batch_types_info.append([BatchType.PORTFOLIO_CREATION, len(base_portfolios), "One job per portfolio"])
    batch_types_to_create.append(BatchType.PORTFOLIO_CREATION)

# Step 03: MRI Import (Base Portfolios only)
if base_portfolios:
    batch_types_info.append([BatchType.MRI_IMPORT, len(base_portfolios), "One job per portfolio"])
    batch_types_to_create.append(BatchType.MRI_IMPORT)

# Step 04: Create Reinsurance Treaties (requires Analysis Table)
if analyses:  # Only need analyses sheet - treaties sheet can be empty
    # Build a set of valid treaty names (may be empty if no treaties defined)
    valid_treaty_names = {t.get('Treaty Name') for t in treaties if t.get('Treaty Name')} if treaties else set()

    # Collect unique treaty-EDM combinations from Analysis Table
    treaty_columns = ['Reinsurance Treaty 1', 'Reinsurance Treaty 2', 'Reinsurance Treaty 3',
                      'Reinsurance Treaty 4', 'Reinsurance Treaty 5']

    for analysis in analyses:
        edm = analysis.get('Database')
        if not edm:
            continue
        for col in treaty_columns:
            treaty_name = analysis.get(col)
            if treaty_name and treaty_name in valid_treaty_names:
                treaty_edm_combinations.add((treaty_name, edm))

    # Always create batch for chaining - even if 0 jobs (empty batch completes immediately)
    job_count = len(treaty_edm_combinations)
    description = f"One job per treaty-EDM combination" if job_count > 0 else "No treaties to create (empty batch for workflow continuity)"
    batch_types_info.append([BatchType.CREATE_REINSURANCE_TREATIES, job_count, description])
    batch_types_to_create.append(BatchType.CREATE_REINSURANCE_TREATIES)

# Step 05: EDM DB Upgrade (Databases sheet + EDM Data Version in Metadata)
if databases and edm_version:
    target_version = edm_version.split('.')[0] if '.' in edm_version else edm_version
    batch_types_info.append([BatchType.EDM_DB_UPGRADE, len(databases), f"One job per database (upgrade to v{target_version})"])
    batch_types_to_create.append(BatchType.EDM_DB_UPGRADE)

# Step 06: GeoHaz (Base Portfolios + Geocode Version in Metadata)
if base_portfolios and geocode_version:
    batch_types_info.append([BatchType.GEOHAZ, len(base_portfolios), f"One job per base portfolio (geocode v{geocode_version})"])
    batch_types_to_create.append(BatchType.GEOHAZ)

# Step 07: Portfolio Mapping (Base Portfolios only)
if base_portfolios:
    batch_types_info.append([BatchType.PORTFOLIO_MAPPING, len(base_portfolios), "One job per base portfolio (SQL mapping)"])
    batch_types_to_create.append(BatchType.PORTFOLIO_MAPPING)

# === STAGE 04: Analysis Execution ===

# Step 01: Analysis (Analysis Table sheet)
if analyses:
    batch_types_info.append([BatchType.ANALYSIS, len(analyses), "One job per analysis"])
    batch_types_to_create.append(BatchType.ANALYSIS)

# === STAGE 05: Grouping ===

# Classify groupings for Grouping and Grouping Rollup batches
if groupings:
    analysis_only_groups, rollup_groups = classify_groupings(configuration_data)

# Step 01: Grouping (analysis-only groups)
if analysis_only_groups:
    batch_types_info.append([BatchType.GROUPING, len(analysis_only_groups), "One job per analysis-only group"])
    batch_types_to_create.append(BatchType.GROUPING)

# Step 02: Grouping Rollup (groups containing other groups)
if rollup_groups:
    batch_types_info.append([BatchType.GROUPING_ROLLUP, len(rollup_groups), "One job per rollup group (groups of groups)"])
    batch_types_to_create.append(BatchType.GROUPING_ROLLUP)

# === STAGE 06: Data Export ===

# Step 01: Export to RDM (requires Export RDM Name and Analysis Table; Groupings optional)
if rdm_name and analyses:
    # One job per analysis/group
    analysis_count = len(analyses)
    group_count = len(groupings) if groupings else 0
    total_jobs = analysis_count + group_count
    batch_types_info.append([BatchType.EXPORT_TO_RDM, total_jobs, f"One job per item ({analysis_count} analyses + {group_count} groups) to '{rdm_name}'"])
    batch_types_to_create.append(BatchType.EXPORT_TO_RDM)

# Display identified batch types
if batch_types_info:
    ux.info("Batch types identified from configuration:")
    ux.table(batch_types_info, headers=["Batch Type", "Job Count", "Description"])
    ux.success(f"✓ Found {len(batch_types_to_create)} batch type(s) to create")
    
    step.log(f"Identified {len(batch_types_to_create)} batch type(s): {', '.join(batch_types_to_create)}")
else:
    ux.warning("⚠ No batch types identified from configuration")
    ux.info("Configuration may not contain required data sheets (Databases, Portfolios, etc.)")
    raise ValueError("No batch types identified in configuration")

## 4) Preview Batch Types

In [ ]:
# Preview all batch types that will be created
# Each batch type shows what jobs will be generated from configuration data

# === Stage 02: Data Extraction ===

# Data Extraction
date_value = metadata.get('Current Date Value', '')
cycle_type = metadata.get('Cycle Type', '')

preview_batch_type(
    BatchType.DATA_EXTRACTION, batch_types_to_create, base_portfolios,
    headers=['Portfolio', 'Import File', 'Account CSV', 'Location CSV'],
    fields=[
        'Portfolio',
        'Import File',
        lambda x: f"Modeling_{date_value}_Moodys_{x.get('Import File', 'XXX')}_Account.csv",
        lambda x: f"Modeling_{date_value}_Moodys_{x.get('Import File', 'XXX')}_Location.csv"
    ],
    notes=[
        'Metadata from configuration file',
        'Portfolio-specific fields from Portfolios sheet',
        f'SQL script: import_files/{{{cycle_type.lower() if cycle_type else "cycle_type"}}}/2_Create_{{Import File}}_Moodys_ImportFile.sql',
        'CSV output to: files/data/'
    ],
    not_needed_msg='Data Extraction batch not needed (no base portfolios in configuration)',
    ux_module=ux, step=step,
    footer='Note: Data Extraction executes SQL scripts locally and exports CSVs to files/data/'
)

# === Stage 03: Data Import ===

# EDM Creation
preview_batch_type(
    BatchType.EDM_CREATION, batch_types_to_create, databases,
    headers=['EDM Name'],
    fields=['Database'],
    notes=['Metadata from configuration file', 'Database-specific fields (Database, Version, EDM_Type, etc.)'],
    not_needed_msg='EDM Creation batch not needed (no databases in configuration)',
    ux_module=ux, step=step
)

# Portfolio Creation  
preview_batch_type(
    BatchType.PORTFOLIO_CREATION, batch_types_to_create, base_portfolios,
    headers=['Portfolio', 'EDM'],
    fields=['Portfolio', 'Database'],
    notes=['Metadata from configuration file', 'Portfolio-specific fields (Portfolio Name, EDM, etc.)'],
    not_needed_msg='Portfolio Creation batch not needed (no base portfolios in configuration)',
    ux_module=ux, step=step
)

# MRI Import
preview_batch_type(
    BatchType.MRI_IMPORT, batch_types_to_create, base_portfolios,
    headers=['Portfolio', 'EDM', 'Import File'],
    fields=['Portfolio', 'Database', 'Import File'],
    notes=['Metadata from configuration file', 'Portfolio-specific fields (Portfolio Name, EDM, Import File, etc.)'],
    not_needed_msg='MRI Import batch not needed (no base portfolios in configuration)',
    ux_module=ux, step=step
)

# Create Reinsurance Treaties
treaty_data = [(t, e) for t, e in sorted(treaty_edm_combinations)] if treaty_edm_combinations else []
preview_batch_type(
    BatchType.CREATE_REINSURANCE_TREATIES, batch_types_to_create, treaty_data,
    headers=['Treaty Name', 'EDM'],
    fields=[lambda x: x[0], lambda x: x[1]],
    notes=['Metadata from configuration file', 'Database (EDM) where treaty will be created', 
           'Treaty-specific fields from Reinsurance Treaties sheet'],
    not_needed_msg='Create Reinsurance Treaties batch not needed (no Analysis Table in configuration)',
    ux_module=ux, step=step
)

# EDM DB Upgrade
target_version = edm_version.split('.')[0] if edm_version and '.' in edm_version else edm_version
preview_batch_type(
    BatchType.EDM_DB_UPGRADE, batch_types_to_create, databases if edm_version else [],
    headers=['EDM Name', 'Target Version'],
    fields=[lambda x: x.get('Database', 'N/A'), lambda x: target_version],
    notes=['Metadata from configuration file', 'Database-specific fields from Databases sheet',
           'target_edm_version: The version to upgrade to'],
    not_needed_msg='EDM DB Upgrade batch not needed (no databases or EDM Data Version not specified)',
    ux_module=ux, step=step,
    extra_info=f"Target EDM Data Version: {target_version}" if target_version else None
)

# GeoHaz
preview_batch_type(
    BatchType.GEOHAZ, batch_types_to_create, base_portfolios if geocode_version else [],
    headers=['Portfolio', 'EDM', 'Geocode Version'],
    fields=[lambda x: x.get('Portfolio', 'N/A'), lambda x: x.get('Database', 'N/A'), lambda x: geocode_version],
    notes=['Metadata from configuration file', 'Portfolio-specific fields from Portfolios sheet',
           'geocode_version: The geocode version to use'],
    not_needed_msg='GeoHaz batch not needed (no base portfolios or Geocode Version not specified)',
    ux_module=ux, step=step,
    extra_info=f"Geocode Version: {geocode_version}" if geocode_version else None
)

# Portfolio Mapping
preview_batch_type(
    BatchType.PORTFOLIO_MAPPING, batch_types_to_create, base_portfolios,
    headers=['Portfolio', 'EDM', 'Import File'],
    fields=['Portfolio', 'Database', 'Import File'],
    notes=['Metadata from configuration file', 'Portfolio-specific fields from Portfolios sheet',
           'SQL script: 2b_Query_To_Create_Sub_Portfolios_{Import File}_RMS_BackEnd.sql'],
    not_needed_msg='Portfolio Mapping batch not needed (no base portfolios in configuration)',
    ux_module=ux, step=step,
    footer='Note: Portfolio Mapping executes SQL scripts locally (not submitted to Moody\'s)'
)

# === Stage 04: Analysis Execution ===

# Analysis
preview_batch_type(
    BatchType.ANALYSIS, batch_types_to_create, analyses,
    headers=['Analysis Name', 'Portfolio', 'EDM', 'Analysis Profile'],
    fields=[lambda x: x.get('Analysis Name', 'N/A'), lambda x: x.get('Portfolio', 'N/A'),
            lambda x: x.get('Database', 'N/A'), lambda x: trunc(x.get('Analysis Profile', 'N/A'), 30)],
    notes=['Metadata from configuration file', 
           'Analysis-specific fields (Name, Portfolio, Database, Profiles, Treaties, Tags)'],
    not_needed_msg='Analysis batch not needed (no analyses in configuration)',
    ux_module=ux, step=step, limit=10
)

# === Stage 05: Grouping ===

# Grouping (Analysis-only)
preview_batch_type(
    BatchType.GROUPING, batch_types_to_create, analysis_only_groups,
    headers=['Group Name', '# Analyses', 'Analyses (Preview)'],
    fields=[lambda x: x.get('Group_Name', 'N/A'), lambda x: len(x.get('items', [])),
            lambda x: list_preview(x.get('items', []), 3)],
    notes=['Metadata from configuration file', 'Group_Name: Name of the group',
           'items: List of analysis names to group together'],
    not_needed_msg='Grouping batch not needed (no analysis-only groups in configuration)',
    ux_module=ux, step=step, limit=10
)

# Grouping Rollup
group_names_set = {g.get('Group_Name') for g in groupings if g.get('Group_Name')}
preview_batch_type(
    BatchType.GROUPING_ROLLUP, batch_types_to_create, rollup_groups,
    headers=['Group Name', '# Group Refs', '# Analysis Refs', 'Items (Preview)'],
    fields=[lambda x: x.get('Group_Name', 'N/A'),
            lambda x: len([i for i in x.get('items', []) if i in group_names_set]),
            lambda x: len([i for i in x.get('items', []) if i not in group_names_set]),
            lambda x: list_preview(x.get('items', []), 3)],
    notes=['Metadata from configuration file', 'Group_Name: Name of the rollup group',
           'items: List of group names AND/OR analysis names to include'],
    not_needed_msg='Grouping Rollup batch not needed (no groups of groups in configuration)',
    ux_module=ux, step=step, limit=10,
    warning='IMPORTANT: Grouping Rollup jobs can only run AFTER the Grouping batch completes.'
)

# === Stage 06: Data Export ===

# Export to RDM (uses special function due to unique structure)
preview_export_to_rdm(
    BatchType.EXPORT_TO_RDM, batch_types_to_create, rdm_name, analyses, groupings,
    ux_module=ux, step=step
)

## 4.5) Check for Existing Batches

In [ ]:
# Check if batches already exist for this configuration
ux.header("Existing Batch Check")

# Get all existing batches for this configuration (exclude COMPLETED and CANCELLED)
existing_batches = get_batches_for_configuration(
    configuration_id=config_id,
    exclude_statuses=['COMPLETED', 'CANCELLED']
)

if existing_batches:
    # Display warning
    ux.warning(f"⚠ Found {len(existing_batches)} existing batch(es) for this configuration:")
    
    # Show existing batches
    batch_display = []
    for batch in existing_batches:
        batch_display.append([
            batch['batch_type'],
            batch['id'],
            batch['status'],
            int(batch['job_count']),
            batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')
        ])
    
    ux.table(batch_display, headers=["Batch Type", "Batch ID", "Status", "Jobs", "Created"])
    
    # Give user options
    print("\nWhat would you like to do?")
    print("  1) RECREATE - Delete existing batches and create new ones")
    print("  2) SKIP     - Keep existing batches and skip creation")
    
    user_choice = input("\nEnter choice (1 or 2): ").strip()
    
    if user_choice == "1":
        # User wants to recreate - delete existing batches
        ux.subheader("Deleting Existing Batches")
        
        for batch in existing_batches:
            batch_id = batch['id']
            batch_type = batch['batch_type']
            
            try:
                delete_batch(batch_id)
                ux.success(f"✓ Deleted batch: {batch_type} (ID={batch_id})")
                step.log(f"Deleted existing batch: {batch_type} (ID={batch_id})")
            except Exception as e:
                ux.error(f"✗ Failed to delete batch {batch_id}: {str(e)}")
                raise
        
        ux.success(f"\n✓ Deleted {len(existing_batches)} existing batch(es)")
        ux.info("Proceeding with batch creation...")
        
    elif user_choice == "2":
        # User wants to skip creation
        ux.info("Keeping existing batches - skipping batch creation")
        step.log("User chose to keep existing batches, skipping creation")
        
        # Set created_batches to existing batches for display purposes
        created_batches = {b['batch_type']: b['id'] for b in existing_batches}
        
        # Calculate total jobs from existing batches
        total_jobs = sum(int(b['job_count']) for b in existing_batches)
        
        # Jump to summary section (skip creation)
        ux.info("\nSkipping to batch summary...")
        
    else:
        ux.error(f"✗ Invalid choice: '{user_choice}'")
        ux.info("Please enter '1' to recreate or '2' to skip")
        raise ValueError(f"Invalid user choice: {user_choice}")
        
else:
    ux.success("✓ No existing batches found - ready to create new batches")
    step.log("No existing batches found for this configuration")

## 5) Create Batches

In [ ]:
# Create batches for identified batch types
# Skip if user chose to keep existing batches
if 'created_batches' in locals() and created_batches:
    ux.header("Batch Creation")
    ux.info("Using existing batches (creation skipped)")
else:
    ux.header("Batch Creation")
    
    # Confirm with user
    batch_summary = ", ".join(batch_types_to_create)
    ux.info(f"Ready to create batches: {batch_summary}")
    proceed = ux.yes_no("Create these batches?")
    
    if not proceed:
        ux.info("Batch creation cancelled by user")
        step.log("User cancelled batch creation")
        raise SystemExit("User cancelled batch creation")
    
    # Create batches
    created_batches = {}
    
    for batch_type in batch_types_to_create:
        ux.subheader(f"Creating batch: {batch_type}")
        
        # Create batch (this will create jobs atomically)
        batch_id = create_batch(
            batch_type=batch_type,
            configuration_id=config_id,
            step_id=step.step_id
        )
        
        # Store batch ID (convert to int to avoid numpy types)
        created_batches[batch_type] = int(batch_id)
        
        # Get job count for this batch
        job_count_result = execute_query(
            "SELECT COUNT(*) as count FROM irp_job WHERE batch_id = %s",
            (batch_id,)
        )
        job_count = int(job_count_result.iloc[0]['count'])
        
        ux.success(f"✓ Batch created: ID={batch_id}")
        ux.info(f"  Jobs created: {job_count}")
        
        step.log(f"Created batch '{batch_type}': ID={batch_id}, Jobs={job_count}")
    
    ux.success(f"\n✓ All batches created successfully ({len(created_batches)} total)")

## 6) Display Batch Summary

In [ ]:
# Display summary of all created batches
ux.header("Batch Summary")

try:
    # Get batch details
    batch_ids = list(created_batches.values())
    
    if batch_ids:
        # Build query to get all batches
        placeholders = ', '.join(['%s'] * len(batch_ids))
        batch_query = f"""
            SELECT 
                b.id,
                b.batch_type,
                b.status,
                b.created_ts,
                COUNT(j.id) as job_count
            FROM irp_batch b
            LEFT JOIN irp_job j ON b.id = j.batch_id
            WHERE b.id IN ({placeholders})
            GROUP BY b.id, b.batch_type, b.status, b.created_ts
            ORDER BY b.created_ts
        """
        
        batch_results = execute_query(batch_query, tuple(batch_ids))
        
        # Display batch information
        batch_rows = []
        total_jobs = 0
        
        for _, batch in batch_results.iterrows():
            batch_rows.append([
                batch['batch_type'],
                batch['id'],
                batch['status'],
                int(batch['job_count']),
                batch['created_ts'].strftime('%Y-%m-%d %H:%M:%S')
            ])
            total_jobs += int(batch['job_count'])
        
        ux.table(batch_rows, headers=["Batch Type", "Batch ID", "Status", "Jobs", "Created"])
        
        ux.info(f"\nTotal batches: {len(batch_ids)}")
        ux.info(f"Total jobs: {total_jobs}")
        
        step.log(f"Batch summary: {len(batch_ids)} batches, {total_jobs} total jobs")
    
except Exception as e:
    ux.error(f"✗ Failed to display batch summary: {str(e)}")
    # Don't fail step, this is just display
    step.log(f"Warning: Failed to display batch summary: {str(e)}", level="WARNING")

## 7) Preview Job Configurations

In [ ]:
# Preview job configurations for created batches
# Order matches step execution: Stage 02 → Stage 03 → Stage 04 → Stage 05 → Stage 06
ux.header("Job Configuration Preview")

# Define the order of batch types to preview (matches execution order)
BATCH_PREVIEW_ORDER = [
    # Stage 02: Data Extraction
    BatchType.DATA_EXTRACTION,
    # Stage 03: Data Import
    BatchType.EDM_CREATION,
    BatchType.PORTFOLIO_CREATION,
    BatchType.MRI_IMPORT,
    BatchType.CREATE_REINSURANCE_TREATIES,
    BatchType.EDM_DB_UPGRADE,
    BatchType.GEOHAZ,
    BatchType.PORTFOLIO_MAPPING,
    # Stage 04: Analysis Execution
    BatchType.ANALYSIS,
    # Stage 05: Grouping
    BatchType.GROUPING,
    BatchType.GROUPING_ROLLUP,
    # Stage 06: Data Export
    BatchType.EXPORT_TO_RDM,
]

try:
    for batch_type in BATCH_PREVIEW_ORDER:
        if batch_type in created_batches:
            batch_id = created_batches[batch_type]
            display_job_preview(batch_id, batch_type, ux, limit=5)
    
    step.log("Job configuration preview displayed")
    
except Exception as e:
    ux.error(f"✗ Failed to preview job configurations: {str(e)}")
    # Don't fail step, this is just display
    step.log(f"Warning: Failed to preview jobs: {str(e)}", level="WARNING")

## 8) Complete Step Execution

In [ ]:
# Complete step execution
ux.header("Step Completion")

# Prepare output data
output_data = {
    'configuration_id': config_id,
    'batches': created_batches,  # {batch_type: batch_id}
    'batch_types_created': batch_types_to_create,
    'total_job_count': total_jobs
}

# Complete the step
step.complete(output_data)

ux.success("\n" + "="*60)
ux.success("✓ BATCHES CREATED SUCCESSFULLY")
ux.success("="*60)
ux.info(f"\nCreated {len(created_batches)} batch(es) with {total_jobs} total job(s)")
ux.info("Batches are in INITIATED status and ready for submission")
ux.info("\nNext: Stage 02 will handle batch submission and job monitoring")